In [2]:
import pandas as pd
from pathlib import Path

DATA_PATH = Path('/mnt/wsl/PHYSICALDRIVE1/data/unsplash')
DATA_PATH.mkdir(exist_ok=True, parents=True)

unsplash_dataset_path = "/home/andras/projects/bipolaroid/unsplash-research-dataset-lite-latest/photos.tsv000"
unsplash_dataset = pd.read_csv(unsplash_dataset_path, sep="\t")
unsplash_dataset.sort_values(by="stats_views", ascending=False, inplace=True)
unsplash_dataset

,photo_id,photo_url,photo_image_url,photo_submitted_at,photo_featured,photo_width,photo_height,photo_aspect_ratio,photo_description,photographer_username,...,photo_location_country,photo_location_city,stats_views,stats_downloads,ai_description,ai_primary_landmark_name,ai_primary_landmark_latitude,ai_primary_landmark_longitude,ai_primary_landmark_confidence,blur_hash
3690,XFmznQhx9lM,https://unsplash.com/photos/XFmznQhx9lM,https://images.unsplash.com/photo-156347321301...,2019-07-18 18:07:14.031684,t,4443,2962,1.50,Fall color in the countryside of Eastern Washi...,timothyeberly,...,NaN,NaN,2978748547,304950,orange leaf trees,NaN,NaN,NaN,NaN,LBJPSa4o0hW?pI4;-.R*E459O?sk
6103,YDNvydD1jAY,https://unsplash.com/photos/YDNvydD1jAY,https://images.unsplash.com/photo-149034936815...,2017-03-24 09:56:57.505262,t,4500,3000,1.50,Flowers in spring,maartendeckers,...,Belgium,NaN,2722857886,416983,"pink, yellow and brown petaled flowers",NaN,NaN,NaN,NaN,LQJInG*JMyIm^ROpxbNFyCNGnln4
3389,4oovIxttThA,https://unsplash.com/photos/4oovIxttThA,https://images.unsplash.com/photo-1560850038-f...,2019-06-18 09:36:35.94311,t,5025,3141,1.60,NaN,a8ka,...,NaN,NaN,2190084956,253730,aerial view of houses near ocean,NaN,NaN,NaN,NaN,LaCt8}~BwNIpozoLofofWBWBaef6
18789,BkR842UVXqk,https://unsplash.com/photos/BkR842UVXqk,https://images.unsplash.com/photo-1558816280-d...,2019-05-25 20:32:08.153319,t,4000,6000,0.67,NaN,olenkasergienko,...,NaN,NaN,1934025254,294785,pink petaled flower,NaN,NaN,NaN,NaN,LA71AxX50_xHt7j[S1ju0_nm^8NZ
21742,GRLN5FC4cLg,https://unsplash.com/photos/GRLN5FC4cLg,https://images.unsplash.com/photo-1552300977-c...,2019-03-11 10:50:25.9311,t,2992,3992,0.75,NaN,turner_imagery,...,NaN,NaN,1916027735,306073,high angle photography of cliff,NaN,NaN,NaN,NaN,LKCr=#~VNat7X-%M%1j?9tNbxaay
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3272,5zsw1PjXg8k,https://unsplash.com/photos/5zsw1PjXg8k,https://images.unsplash.com/photo-142372152343...,2015-02-12 06:12:09.092905,f,2448,3264,0.75,NaN,melissaaskew,...,NaN,NaN,14088,812,waterfalls in the middle of the forest,NaN,NaN,NaN,NaN,LG8#NK.84m4mt6f#RjkD9EM_%N-=
3273,gqa-fnYASIQ,https://unsplash.com/photos/gqa-fnYASIQ,https://images.unsplash.com/photo-142302719730...,2015-02-04 05:19:59.869141,f,5086,3391,1.50,NaN,wilstewart3,...,NaN,NaN,13384,858,a street sign sitting on the side of a body of...,NaN,NaN,NaN,NaN,"LvKKi,RjM{j[_NWBWBfk5EoLoeaz"
12956,Cq62qvCW8bM,https://unsplash.com/photos/Cq62qvCW8bM,https://images.unsplash.com/photo-142245228993...,2015-01-28 13:38:18.071331,f,4896,3264,1.50,NaN,kseny,...,NaN,NaN,13068,1054,man in black shirt and blue pants sitting on b...,NaN,NaN,NaN,NaN,LGEW2ko~M{%N0;ofnhRkwvozt8of
17074,Py8vZdCw35U,https://unsplash.com/photos/Py8vZdCw35U,https://images.unsplash.com/photo-142034363140...,2015-01-04 03:54:41.031772,f,2320,1553,1.49,NaN,mrbrodeur,...,NaN,NaN,12617,581,man in black jacket standing on brown sand und...,NaN,NaN,NaN,NaN,LuI~ZRogaxR*0?Rjofj[Mxs.a|fP


In [3]:
import concurrent.futures
import requests
from tqdm import tqdm
from typing import List

progress = tqdm(total=len(unsplash_dataset))

def download_image(row):
    filename = DATA_PATH / f"{row['photo_id']}.jpg"
    for _ in range(10):
        try:
            response = requests.get(row["photo_image_url"], timeout=10)
            with open(filename, "wb") as f:
                f.write(response.content)
            with progress.get_lock():
                progress.update(1)
            break
        except Exception as e:
            print(f"Error downloading {row['photo_id']}: {e}")

with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
    futures: List[concurrent.futures.Future] = []
    for row in unsplash_dataset.iterrows():
        row = row[1]
        future = executor.submit(download_image, row)
        futures.append(future)

    progress.display()
    concurrent.futures.wait(futures)
progress.close()

  0%|          | 113/25000 [00:30<1:02:36,  6.62it/s]

Error downloading toPRrcyAIUY: HTTPSConnectionPool(host='images.unsplash.com', port=443): Read timed out.


  1%|          | 184/25000 [00:48<1:19:49,  5.18it/s]

Error downloading t7YycgAoVSw: HTTPSConnectionPool(host='images.unsplash.com', port=443): Read timed out.


  1%|          | 219/25000 [00:58<1:40:16,  4.12it/s]

...

 47%|████▋     | 11655/25000 [52:08<1:05:47,  3.38it/s]

Error downloading PgBTaq-AgVI: HTTPSConnectionPool(host='images.unsplash.com', port=443): Read timed out.


 62%|██████▏   | 15477/25000 [1:10:12<47:47,  3.32it/s]  

Error downloading zS_b76LrEL8: HTTPSConnectionPool(host='images.unsplash.com', port=443): Read timed out.


 70%|██████▉   | 17470/25000 [1:19:44<19:51,  6.32it/s]  